# Exampville Notebook
This Jupyter Notebook demostrates a part of a typical workflow for a transportation model.  


## Simulating the Travel Survey

We begin by generating a simulated travel survey for the people of Exampville.  The latest version of Larch includes a helpful simulation tool, which allows us to scale the size of Examplville to the amount of time we want to spend processing the example (and the power of our computer).  Although we can expect better model estimates when the sample size is larger, we're mostly concerned with the technical operation of Larch and not with getting the best model fit we can, so we'll simulate only a small amount of data.

In [1]:
import larch, numpy, pandas, os
from larch.roles import P,X

from larch.examples.exampville import builder
numpy.set_printoptions(linewidth=160)

nZones = 15
transit_scope = (4,15)
n_HH = 2000

directory='/tmp/exampville'  # if on Windows, change this to a suitable temp dir
if not os.path.exists(directory):
    os.makedirs(directory)
os.chdir(directory)

directory, omx, f_hh, f_pp, f_tour = builder(
    nZones=nZones, 
    transit_scope=transit_scope, 
    n_HH=n_HH,
    directory=directory,
)

f_hh.export_idco(os.path.join(directory,'exampville_hh.csv'))
f_pp.export_idco(os.path.join(directory,'exampville_person.csv'))
f_tour.export_idco(os.path.join(directory,'exampville_tours.csv'))

[10:59:10 PM]root: Connected log to stream <ipykernel.iostream.OutStream object at 0x104d6fc88>
[10:59:10 PM]larch.exampville: EXAMPVILLE Builder (Year 1)
[10:59:10 PM]larch.exampville:   simulating a survey of 2000 households
[10:59:10 PM]larch.exampville:   traveling among 15 travel analysis zones
[10:59:10 PM]larch.exampville: Zones
[10:59:10 PM]larch.exampville: Skims
[10:59:10 PM]larch.exampville: HHs
[10:59:10 PM]larch.exampville: People
[10:59:10 PM]larch.exampville: Tours
[10:59:10 PM]larch.exampville: Choice Probability
[10:59:10 PM]larch.exampville: Choices
[10:59:10 PM]larch.exampville: Output
[10:59:11 PM]larch.exampville: EXAMPVILLE Completed Builder (Year 1)
[10:59:11 PM]larch.exampville:    SKIMS  : /tmp/exampville/exampville.omx
[10:59:11 PM]larch.exampville:    HHs    : /tmp/exampville/exampville_hh.h5
[10:59:11 PM]larch.exampville:    Persons: /tmp/exampville/exampville_person.h5
[10:59:11 PM]larch.exampville:    Tours  : /tmp/exampville/exampville_tours.h5


In [2]:
omx

<larch.OMX> /tmp/exampville/exampville.omx
 |  shape:(15, 15)
 |  data:
 |    AUTO_TIME (float64)
 |    DIST      (float64)
 |    RAIL_FARE (float64)
 |    RAIL_TIME (float64)
 |  lookup:
 |    EMPLOYMENT    (15 float64)
 |    EMP_NONRETAIL (15 float64)
 |    EMP_RETAIL    (15 float64)
 |    TAZID         (15 int64)

In [3]:
f_hh

<larch.DT> /tmp/exampville/exampville_hh.h5
 |  > file is opened for read/write <
 |  nCases: 2000
 |  nAlts: <missing>
 |  idco:
 |    HHSIZE  	int64  
 |    HOMETAZ 	int64  
 |    INCOME  	int64  

In [4]:
f_pp

<larch.DT> /tmp/exampville/exampville_person.h5
 |  > file is opened for read/write <
 |  nCases: 3462
 |  nAlts: <missing>
 |  idco:
 |    AGE         	int64  
 |    HHID        	int64  
 |    N_OTHERTOURS	int64  
 |    N_TOTALTOURS	int64  
 |    N_WORKTOURS 	int64  
 |    WORKS       	int64  

In [5]:
f_tour

<larch.DT> /tmp/exampville/exampville_tours.h5
 |  > file is opened for read/write <
 |  nCases: 6123
 |  nAlts: <missing>
 |  idco:
 |    DTAZ    	int64  
 |    HHID    	int64  
 |    PERSONID	int64  
 |    TOURMODE	int64  
 |    TOURPURP	int64  

## Compiling Mode Choice Data

The Exampville data output contains a set of files similar to what we might find for a real travel survey: network skims, and tables of households, persons, and tours.  We'll need to merge these tables to create a composite dataset for mode choice model estimation.

In [6]:
flog = larch.logging.flogger('ModeData')


### MODE CHOICE DATA

# Define numbers and names for modes
DA = 1
SR = 2
Walk = 3
Bike = 4
Transit = 5

d = larch.DT() 
# By omitting a filename here, we create a temporary HDF5 store.

d.set_alternatives( [DA,SR,Walk,Bike,Transit], ['DA','SR','Walk','Bike','Transit'] )

flog('merging survey datasets')
d.new_caseids( f_tour.caseids() )
d.merge_into_idco(f_tour, "caseid")
d.merge_into_idco(f_pp, "PERSONID")
d.merge_into_idco(f_hh, "HHID")

flog('merging skims')
# Create a new variables with zero-based home TAZ numbers
d.new_idco("HOMETAZi", "HOMETAZ-1", dtype=int)
d.new_idco("DTAZi", "DTAZ-1", dtype=int)

# Pull in plucked data from Matrix file
d.pluck_into_idco(omx, "HOMETAZi", "DTAZi")
# This command is new as of Larch 3.3.15
# It loads all the matrix DATA from an OMX based on OTAZ and DTAZ 
# columns that already exist in the DT

flog('prep data')
d.choice_idco = {
    DA: 'TOURMODE==1',
    SR: 'TOURMODE==2',
    Walk: 'TOURMODE==3',
    Bike: 'TOURMODE==4',
    Transit: 'TOURMODE==5',
}
# Alternately:   d.choice_idco = {i:'TOURMODE=={}'.format(i) for i in [1,2,3,4,5]}



d.avail_idco = {
    DA: '(AGE>=16)',
    SR: '1',
    Walk: 'DIST<=3',
    Bike: 'DIST<=15',
    Transit: 'RAIL_TIME>0',
}

# Let's define some variables for clarity.
d.new_idco("WALKTIME", "DIST / 2.5 * 60 * (DIST<=3)") # 2.5 mph, 60 minutes per hour, max 3 miles
d.new_idco("BIKETIME", "DIST / 12 * 60 * (DIST<=15)")  # 12 mph, 60 minutes per hour, max 15 miles
d.new_idco("CARCOST", "DIST * 0.20")  # 20 cents per mile

d.info(1)

[10:59:11 PM]larch: merging survey datasets
[10:59:11 PM]larch: merging skims
[10:59:11 PM]larch: prep data


<larch.DT> tmplryxra7g.h5f
┌────────────┬───────┬───────┬────────────────────────────────────┐
│Variable    │dtype  │Shape  │Original Source                     │
├────────────┼───────┼───────┼────────────────────────────────────┤
╞ idco ═══════════════════════════════════════════════════════════╡
│AGE         │int64  │(6123,)│/tmp/exampville/exampville_person.h5│
│AUTO_TIME   │float64│(6123,)│/tmp/exampville/exampville.omx      │
│BIKETIME    │float64│(6123,)                                     │
│CARCOST     │float64│(6123,)                                     │
│DIST        │float64│(6123,)│/tmp/exampville/exampville.omx      │
│DTAZ        │int64  │(6123,)│/tmp/exampville/exampville_tours.h5 │
│DTAZi       │int64  │(6123,)                                     │
│HHID        │int64  │(6123,)│/tmp/exampville/exampville_tours.h5 │
│HHSIZE      │int64  │(6123,)│/tmp/exampville/exampville_hh.h5    │
│HOMETAZ     │int64  │(6123,)│/tmp/exampville/exampville_hh.h5    │
│HOMETAZi    │int64  │(6123,)                                     │
│INCOME      │int64  │(6123,)│/tmp/exampville/exampville_hh.h5    │
│N_OTHERTOURS│int64  │(6123,)│/tmp/exampville/exampville_person.h5│
│N_TOTALTOURS│int64  │(6123,)│/tmp/exampville/exampville_person.h5│
│N_WORKTOURS │int64  │(6123,)│/tmp/exampville/exampville_person.h5│
│PERSONID    │int64  │(6123,)│/tmp/exampville/exampville_tours.h5 │
│RAIL_FARE   │float64│(6123,)│/tmp/exampville/exampville.omx      │
│RAIL_TIME   │float64│(6123,)│/tmp/exampville/exampville.omx      │
│TOURMODE    │int64  │(6123,)│/tmp/exampville/exampville_tours.h5 │
│TOURPURP    │int64  │(6123,)│/tmp/exampville/exampville_tours.h5 │
│WALKTIME    │float64│(6123,)                                     │
│WORKS       │int64  │(6123,)│/tmp/exampville/exampville_person.h5│
│caseid      │int64  │(6123,)│/tmp/exampville/exampville_tours.h5 │
╞ idca ═══════════════════════════════════════════════════════════╡
│_avail_     │<stack>│<stack>                                     │
│_choice_    │<stack>│<stack>                                     │
└────────────┴───────┴───────┴────────────────────────────────────┘

## Mode Choice Model Estimation

In [7]:
flog = larch.logging.flogger('ModeModel')

d.exclude_idco("TOURPURP != 1")
flog("nCases={}", d.nCases())

### MODE CHOICE MODEL

m = larch.Model(d)
m.title = "Exampville Work Tour Mode Choice v1"

# In order to create a shadow (alias) parameter, the source parameter must already exist,
# so we create it explicitly.
m.parameter("InVehTime")

# Then we can create the shadow, as a multiple of the original parameter.
m.shadow_parameter.NonMotorTime = P.InVehTime * 2



m.utility.co[DA] = (
    + P.InVehTime * X.AUTO_TIME
    + P.Cost * X.CARCOST # dollars per mile
)

m.utility[SR] = (
    + P.ASC_SR
    + P.InVehTime * X.AUTO_TIME
    + P.Cost * (X.CARCOST * 0.5) # dollars per mile, half share
    + P("HighInc:SR") * X("INCOME>75000")
)

m.utility[Walk] = (
    + P.ASC_Walk
    + P.NonMotorTime * X.WALKTIME 
    + P("HighInc:Walk") * X("INCOME>75000")
)

m.utility[Bike] = (
    + P.ASC_Bike
    + P.NonMotorTime * X.BIKETIME
    + P("HighInc:Bike") * X("INCOME>75000")
)

m.utility[Transit] = (
    + P.ASC_Transit
    + P.InVehTime * X.RAIL_TIME
    + P.Cost * X.RAIL_FARE
    + P("HighInc:Transit") * X("INCOME>75000")
)

Car = m.new_nest('Nest:Car', children=[DA,SR])
NonMotor = m.new_nest('Nest:NonMotor', children=[Walk,Bike])
Motor = m.new_nest('Nest:Motorized', children=[Car,Transit])

from larch.util.categorize import Categorizer

m.parameter_groups = (
    Categorizer("Level of Service",
        ".*Time.*",
        ".*Cost.*",
    ),
    Categorizer("Alternative Specific Constants",
        "ASC.*",
    ),
    Categorizer("Income",
        ".*HighInc.*",
        ".*LowInc.*",
    ),
    Categorizer("Logsum Parameters",
        "Nest.*",
    ),
)

m.maximize_loglike()


[10:59:11 PM]larch: nCases=1897
[10:59:11 PM]larch.Model: Model.setUp...
[10:59:11 PM]larch.Model: Model.setUp complete
[10:59:11 PM]larch.Model: Using <larch.util.optimize.OptimizeTechnique SLSQP>
[10:59:12 PM]larch.Model: Optimization terminated successfully. [SLSQP]
[10:59:12 PM]larch.Model: Ending leg (success) using SLSQP at [-0.11274266  0.74170259  0.86969656  0.84439806 -0.3407393  -1.39431543 -1.7562362   0.73177238 -0.79580814 -1.6667516  -1.0525934  -1.67706804 -1.24100481]
[10:59:12 PM]larch.Model: Preliminary Results
+-----------------+---------------+----------+------+----------+
 Parameter         Estimated Value Std Error  t-Stat Null Value 
+-----------------+---------------+----------+------+----------+
                                                                
= Level of Service =============================================
 InVehTime         -0.1127          nan        nan    0         
 NonMotorTime      -0.2255         = InVehTime * 2              
 Cost    

          messages: Optimization terminated successfully. [SLSQP]:
                  |     SLSQP:Optimization terminated successfully.
              ctol: 1.7239307248605795e-10
               fun: 963.8426415309937
  installed_memory: '24.0 GiB'
               jac: array([ -3.64328217e-04,  -2.93931314e-04,  -2.03849123e-05,  -3.39838212e-05,   1.13263369e-05,  -1.04499335e-04,  -4.78475259e-05,  -1.19731658e-06,
                  |          3.87914401e-06,  -1.88932476e-05,  -5.70381211e-06,  -4.49764641e-06,  -9.45794384e-06,   0.00000000e+00])
           loglike: -963.8426415309937
      loglike_null: -2592.698004024909
           message: 'Optimization terminated successfully. [SLSQP]'
               nit: 42
             niter: [('SLSQP', 42)]
 peak_memory_usage: '132.6015625 MiB'
             stats: <larch.core.runstats, success in 0:00.49>
            status: 0
           success: True
                 x: array([-0.11274266,  0.74170259,  0.86969656,  0.84439806, -0.3407393 , -1

In [8]:
## Save results to a report
m.xhtml_report(filename=os.path.join(directory,'mode_choice_model.html'), cats='**')



## View Mode Choice Model Results
# Here we lump together a bunch of report sections into a single report that will
# display neatly in a jupyter notebook.
sections = ['params','ll',
 'nesting_tree',
 'latest','UTILITYSPEC','PROBABILITYSPEC',
 'DATA', 
 'excludedcases','NOTES','options',
 'possible_overspecification']
m.jupyter(*sections)

Model Parameter Estimates
┌─────────────────┬───────────────┬──────────┬──────┬──────────┐
│Parameter        │Estimated Value│Std Error │t-Stat│Null Value│
├─────────────────┼───────────────┼──────────┼──────┼──────────┤
│       │         │               │          │      │          │
╞ Level of Service ════════════════════════════════════════════╡
│InVehTime        │-0.1127        │ 0.01356  │-8.31 │ 0        │
│NonMotorTime     │-0.2255        │= InVehTime * 2             │
│Cost             │-0.3407        │ 0.1812   │-1.88 │ 0        │
╞ Alternative Specific Constants ══════════════════════════════╡
│ASC_SR           │-1.394         │ 0.9541   │-1.46 │ 0        │
│ASC_Walk         │ 0.7318        │ 0.3015   │ 2.43 │ 0        │
│ASC_Bike         │-1.667         │ 0.2739   │-6.09 │ 0        │
│ASC_Transit      │-1.677         │ 0.4977   │-3.37 │ 0        │
╞ Income ══════════════════════════════════════════════════════╡
│HighInc│SR       │-1.756         │ 1.29     │-1.36 │ 0        │
│       │Walk     │-0.7958        │ 0.4189   │-1.90 │ 0        │
│       │Bike     │-1.053         │ 0.4631   │-2.27 │ 0        │
│       │Transit  │-1.241         │ 0.4508   │-2.75 │ 0        │
╞ Logsum Parameters ═══════════════════════════════════════════╡
│Nest   │Car      │ 0.7417        │ 0.5384   │-0.48 │ 1        │
│       │NonMotor │ 0.8697        │ 0.1835   │-0.71 │ 1        │
│       │Motorized│ 0.8444        │ 0.2487   │-0.63 │ 1        │
└─────────────────┴───────────────┴──────────┴──────┴──────────┘

Model Estimation Statistics
┌──────────────────────────────────┬─────────┬────────┐
│Statistic                         │Aggregate│Per Case│
├──────────────────────────────────┼─────────┼────────┤
│Number of Cases                   │       1897       │
│Log Likelihood at Convergence     │-963.84  │-0.51   │
│Log Likelihood at Null Parameters │-2592.70 │-1.37   │
│Rho Squared w.r.t. Null Parameters│      0.628       │
└──────────────────────────────────┴─────────┴────────┘

Latest Estimation Run Statistics
┌────────────────────────────────────┬─────────────────────────────────────────────┐
│Estimation Date                     │Friday, October 28 2016, 10:59:11 PM         │
│Results                             │success                                      │
│Message                             │Optimization terminated successfully. [SLSQP]│
│Optimization Method                 │SLSQP                                        │
│Number of Iterations                │42                                           │
│Running Time│Total                  │0.486 seconds                                │
│            │setup                  │0:00.08                                      │
│            │null_likelihood        │0:00                                         │
│            │weight choice rebalance│0:00                                         │
│            │weight autorescale     │0:00                                         │
│            │optimize:SLSQP         │0:00.33                                      │
│            │parameter covariance   │0:00.08                                      │
│            │cleanup                │0:00                                         │
│Notes                               │did not automatically rescale weights        │
│Computer                            │paladin.thunderdome                          │
│Processor                           │Intel(R) Core(TM) i7-6700K CPU @ 4.00GHz     │
│Number of CPU Cores                 │8                                            │
│Number of Threads Used              │8                                            │
│Installed Memory                    │24.0 GiB                                     │
│Peak Memory Usage                   │132.6015625 MiB                              │
└────────────────────────────────────┴─────────────────────────────────────────────┘

Code,Alternative,Resolved Utility
1,DA,- 0.1127*AUTO_TIME- 0.3407*CARCOST
2,SR,- 1.394- 0.1127*AUTO_TIME- 0.3407*CARCOST*(0.5)- 1.756*INCOME>75000
3,Walk,0.7318- 0.2255*WALKTIME- 0.7958*INCOME>75000
4,Bike,- 1.667- 0.2255*BIKETIME- 1.053*INCOME>75000
5,Transit,- 1.677- 0.1127*RAIL_TIME- 0.3407*RAIL_FARE- 1.241*INCOME>75000
Code,Nest,Resolved Utility
6,Nest:Car,0.7417 * log( exp(Utility[DA]/0.7417) + exp(Utility[SR]/0.7417) )
7,Nest:NonMotor,0.8697 * log( exp(Utility[Walk]/0.8697) + exp(Utility[Bike]/0.8697) )
8,Nest:Motorized,0.8444 * log( exp(Utility[Transit]/0.8444) + exp(Utility[Nest:Car]/0.8444) )
0,ROOT,log( exp(Utility[Nest:Motorized]) + exp(Utility[Nest:NonMotor]) )


Code,Alternative,Resolved Probability
1,DA,exp(Utility[DA]/0.7417)/exp(Utility[Nest:Car]/0.7417) * exp(Utility[Nest:Car]/0.8444)/exp(Utility[Nest:Motorized]/0.8444) * exp(Utility[Nest:Motorized])/exp(Utility[ROOT])
2,SR,exp(Utility[SR]/0.7417)/exp(Utility[Nest:Car]/0.7417) * exp(Utility[Nest:Car]/0.8444)/exp(Utility[Nest:Motorized]/0.8444) * exp(Utility[Nest:Motorized])/exp(Utility[ROOT])
3,Walk,exp(Utility[Walk]/0.8697)/exp(Utility[Nest:NonMotor]/0.8697) * exp(Utility[Nest:NonMotor])/exp(Utility[ROOT])
4,Bike,exp(Utility[Bike]/0.8697)/exp(Utility[Nest:NonMotor]/0.8697) * exp(Utility[Nest:NonMotor])/exp(Utility[ROOT])
5,Transit,exp(Utility[Transit]/0.8444)/exp(Utility[Nest:Motorized]/0.8444) * exp(Utility[Nest:Motorized])/exp(Utility[ROOT])
Code,Alternative,Formulaic Probability
1,DA,exp(Utility[DA]/Nest:Car)/exp(Utility[Nest:Car]/Nest:Car) * exp(Utility[Nest:Car]/Nest:Motorized)/exp(Utility[Nest:Motorized]/Nest:Motorized) * exp(Utility[Nest:Motorized])/exp(Utility[ROOT])
2,SR,exp(Utility[SR]/Nest:Car)/exp(Utility[Nest:Car]/Nest:Car) * exp(Utility[Nest:Car]/Nest:Motorized)/exp(Utility[Nest:Motorized]/Nest:Motorized) * exp(Utility[Nest:Motorized])/exp(Utility[ROOT])
3,Walk,exp(Utility[Walk]/Nest:NonMotor)/exp(Utility[Nest:NonMotor]/Nest:NonMotor) * exp(Utility[Nest:NonMotor])/exp(Utility[ROOT])
4,Bike,exp(Utility[Bike]/Nest:NonMotor)/exp(Utility[Nest:NonMotor]/Nest:NonMotor) * exp(Utility[Nest:NonMotor])/exp(Utility[ROOT])


Code,Alternative,# Avail,# Chosen,Availability Condition
1,DA,1897,1573,(AGE>=16)
2,SR,1897,171,1
3,Walk,662,47,DIST<=3
4,Bike,1882,28,DIST<=15
5,Transit,1245,78,RAIL_TIME>0


,Criteria,Data Source,# Cases Excluded,# Cases Remaining
0,TOURPURP != 1,idco,4226,1897


author,jpn
autocreate_parameters,True
calc_null_likelihood,True
calc_std_errors,True
enforce_bounds,True
enforce_constraints,True
enforce_network_constraints,False
force_finite_diff_grad,False
force_recalculate,False
gradient_diagnostic,0
hessian_diagnostic,0


In [9]:
m.jupyter.choice_distributions

Choice Distributions by Data Attributes
┌─────────────┬───────────┬──────┬──────┬──────┬─────┬───────┬──────┐
│Variable     │Value[s]   │DA    │SR    │Walk  │Bike │Transit│Count │
├─────────────┼───────────┼──────┼──────┼──────┼─────┼───────┼──────┤
│AUTO_TIME    │> 8.6      │83.31%│10.81%│0.00% │0.16%│5.72%  │629.0 │
│             │4.1 to 8.6 │86.88%│9.72% │0.00% │0.77%│2.62%  │648.0 │
│             │< 4.1      │78.39%│6.45% │7.58% │3.55%│4.03%  │620.0 │
│CARCOST      │> 1.3      │84.31%│11.89%│0.00% │0.00%│3.80%  │631.0 │
│             │0.39 to 1.3│86.80%│8.50% │0.00% │0.61%│4.10%  │659.0 │
│             │< 0.39     │77.27%│6.59% │7.74% │3.95%│4.45%  │607.0 │
│CARCOST*(0.5)│> 0.64     │84.31%│11.89%│0.00% │0.00%│3.80%  │631.0 │
│             │0.2 to 0.64│86.80%│8.50% │0.00% │0.61%│4.10%  │659.0 │
│             │< 0.2      │77.27%│6.59% │7.74% │3.95%│4.45%  │607.0 │
│INCOME>75000 │Yes        │94.20%│1.73% │1.60% │0.74%│1.73%  │810.0 │
│             │No         │74.52%│14.44%│3.13% │2.02%│5.89%  │1087.0│
│WALKTIME     │> 37       │85.78%│7.80% │0.00% │2.29%│4.13%  │218.0 │
│             │23 to 37   │84.82%│8.48% │0.45% │2.23%│4.02%  │224.0 │
│             │< 23       │63.64%│4.55% │20.91%│6.36%│4.55%  │220.0 │
│             │= 0        │85.51%│10.12%│0.00% │0.32%│4.05%  │1235.0│
│BIKETIME     │> 31       │84.22%│12.08%│0.00% │0.00%│3.70%  │621.0 │
│             │9.8 to 31  │86.70%│8.56% │0.00% │0.61%│4.13%  │654.0 │
│             │< 9.8      │77.27%│6.59% │7.74% │3.95%│4.45%  │607.0 │
│             │= 0        │93.33%│0.00% │0.00% │0.00%│6.67%  │15.0  │
│RAIL_TIME    │> 7.7      │85.17%│7.93% │0.00% │0.00%│6.91%  │391.0 │
│             │2.6 to 7.7 │83.00%│8.83% │0.00% │1.32%│6.84%  │453.0 │
│             │< 2.6      │75.56%│6.23% │8.73% │4.49%│4.99%  │401.0 │
│             │= 0        │86.04%│11.50%│1.84% │0.61%│0.00%  │652.0 │
│RAIL_FARE    │0.0        │86.04%│11.50%│1.84% │0.61%│0.00%  │652.0 │
│             │1.5        │81.29%│7.71% │2.81% │1.93%│6.27%  │1245.0│
└─────────────┴───────────┴──────┴──────┴──────┴─────┴───────┴──────┘

In [10]:
m.jupyter.datasummary

Utility idCO Data
┌─────────────┬───────┬────────┬─────────┬───────┬─────┬─────────────┬─────────┬────────────┐
│Data         │Mean   │Std.Dev.│Minimum  │Maximum│Zeros│Mean(NonZero)│Positives│Distribution│
├─────────────┼───────┼────────┼─────────┼───────┼─────┼─────────────┼─────────┼────────────┤
│AUTO_TIME    │7.7198 │5.637   │2        │49.678 │0    │7.7198       │1897     │            │
│CARCOST      │0.97796│0.71928 │0.0040437│3.621  │0    │0.97796      │1897     │            │
│1            │1      │0       │1        │1      │0    │1            │1897     │            │
│CARCOST*(0.5)│0.48898│0.35964 │0.0020218│1.8105 │0    │0.48898      │1897     │            │
│INCOME>75000 │0.42699│0.49464 │0        │1      │1087 │1            │810      │            │
│WALKTIME     │10.386 │17.018  │0        │71.245 │1235 │29.763       │662      │            │
│BIKETIME     │23.819 │17.419  │0        │73.871 │15   │24.009       │1882     │            │
│RAIL_TIME    │4.1426 │5.4032  │0        │28.821 │652  │6.312        │1245     │            │
│RAIL_FARE    │0.98445│0.71241 │0        │1.5    │652  │1.5          │1245     │            │
└─────────────┴───────┴────────┴─────────┴───────┴─────┴─────────────┴─────────┴────────────┘
 Graphs are represented as pie charts if the data element has 4 or fewer distinct values.
 Histograms are green if the displayed range truncates some extreme outliers.
 Histograms are orange if the zeros are numerous and have been excluded.

 Utility idCO Data by Alternative
┌───────────┬─────────────┬────────┬────────┬────────┬────────────────────┬──────────────────┬───────────────┬───────┬───────────┬────────────┐
│Alternative│Data         │Filter  │Mean    │Std.Dev.│Minimum             │Maximum           │Mean (Nonzeros)│# Zeros│# Positives│Distribution│
├───────────┼─────────────┼────────┼────────┼────────┼────────────────────┼──────────────────┼───────────────┼───────┼───────────┼────────────┤
│DA         │AUTO_TIME    │Chosen  │7.8347  │5.607   │2.0                 │49.67825818294802 │7.8347         │0      │1573       │            │
│           │             │Unchosen│7.1621  │5.7478  │2.0                 │33.84280990076548 │7.1621         │0      │324        │            │
│           │CARCOST      │Chosen  │1.0037  │0.70675 │0.004043679488065144│3.6210315487992397│1.0037         │0      │1573       │            │
│           │             │Unchosen│0.85277 │0.765   │0.004043679488065144│3.02732395233208  │0.85277        │0      │324        │            │
│SR         │AUTO_TIME    │Chosen  │8.4195  │5.3067  │2.0                 │31.692757725069843│8.4195         │0      │171        │            │
│           │             │Unchosen│7.6505  │5.664   │2.0                 │49.67825818294802 │7.6505         │0      │1726       │            │
│           │CARCOST*(0.5)│Chosen  │0.55414 │0.3506  │0.011827442586893323│1.4774221314319649│0.55414        │0      │171        │            │
│           │             │Unchosen│0.48253 │0.35988 │0.002021839744032572│1.8105157743996199│0.48253        │0      │1726       │            │
│           │INCOME>75000 │Chosen  │0.081871│0.27417 │0.0                 │1.0               │1              │157    │14         │            │
│           │             │Unchosen│0.46118 │0.49849 │0.0                 │1.0               │1              │930    │796        │            │
│Walk       │INCOME>75000 │Chosen  │0.2766  │0.44731 │0.0                 │1.0               │1              │34     │13         │            │
│           │             │Unchosen│0.40325 │0.49055 │0.0                 │1.0               │1              │367    │248        │            │
│           │WALKTIME     │Chosen  │3.8568  │4.4628  │0.4852415385678172  │23.486840213586337│3.8568         │0      │47         │            │
│           │             │Unchosen│31.743  │14.69   │0.4852415385678172  │71.24491157605924 │31.743         │0      │615        │            │
│Bike       │INCOME>75000 │Chosen  │

In [11]:
## Saving the model
m_filename = os.path.join(directory, 'exampville_workmodechoice_v1.larchmodel')
m.save(m_filename)
m_filename

'/tmp/exampville/exampville_workmodechoice_v1.larchmodel'

In [12]:
m2 = larch.Model.load(m_filename)
print(m2)

Exampville Work Tour Mode Choice v1
Model Parameter Estimates
--------------------------------------------------------------------------------------------
Parameter      	InitValue   	FinalValue  	StdError    	t-Stat      	NullValue   
~~ Level of Service ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
InVehTime      	 0          	-0.112743   	 0.0135618  	-8.31328    	 0          
Cost           	 0          	-0.340739   	 0.181214   	-1.88032    	 0          
~~ Alternative Specific Constants ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ASC_SR         	 0          	-1.39432    	 0.954061   	-1.46145    	 0          
ASC_Walk       	 0          	 0.731772   	 0.301454   	 2.42747    	 0          
ASC_Bike       	 0          	-1.66675    	 0.273898   	-6.08529    	 0          
ASC_Transit    	 0          	-1.67707    	 0.497688   	-3.36972    	 0          
~~ Income ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

## Building Mode Choice Logsums

We're going to build the mode choice logsums, and store them in the DT
for use in the destination choice model.  

One important thing to recall is that we applied some exclusion filters to
the DT, to get only work tours (as opposed to all tours).  In order to 
remain consistent, new arrays we add to the DT need to have the same number
of rows as existing DT arrays (and the same number of caseids). 

In [13]:
print(d.nCases())
print(d.nAllCases())

1897
6123


The `nCases` method returns the number of active (non-excluded) cases, while the `nAllCases` method counts _all_ of the cases in the DT, whether they are active or not.

Another concern is that the non-excluded cases are not a contiguous set; they are spread around the list of all cases.  Fortunately, we have a method to extract the current active case indexes, which we will use to expand the logsums we create and push them back into the DT in the correct places.

In [14]:
screen_idx = d.get_screen_indexes()
print(screen_idx)
print(screen_idx.shape)

[   0    2    3 ..., 6112 6116 6119]
(1897,)


In [15]:
flog = larch.logging.flogger('modelogsum')

# First create a new blank array for the logsums.
d.new_idca('MODECHOICELOGSUM', numpy.zeros([d.nAllCases(), nZones], dtype=numpy.float32))

# We will also create a seperate array in memory, to cache the logsums we calculate 
# so that we can push all the calculated values into the DT on disk in one pass at the end.
# Note the in-memory version is sized for nCases, while the disk version used nAllCases.
modechoicelogsums = numpy.zeros([d.nCases(), nZones], dtype=numpy.float32)

m.setUp()
# If the model was just estimated, this is unnecessary; otherwise this is necessary
# to allocate the needed arrays for storing data and results.

# Loop over all TAZ indexes based on the number of zones
for dtazi in range(nZones):
    # First we pull in replacement data from the skims,
    # replacing the data for the actually chosen destination with alternative
    # data from the proposed destination
    d.pluck_into_idco(omx, "HOMETAZi", dtazi, overwrite=True)
    
    # We need to refresh these derived variables too, as they are derived
    # from other data columns we just overloaded...
    d.new_idco("WALKTIME", "DIST / 2.5 * 60 * (DIST<=3)", overwrite=True) # 2.5 mph, 60 minutes per hour, max 3 miles
    d.new_idco("BIKETIME", "DIST / 12 * 60 * (DIST<=15)", overwrite=True)  # 12 mph, 60 minutes per hour, max 15 miles
    d.new_idco("CARCOST", "DIST * 0.20", overwrite=True)  # 20 cents per mile
    
    # Then we will load the relevant data into the Model
    m.provision()
    
    # Then we calculate and extract the logsums using the model and store
    # them in the correct indexes of the relevant column in the output array
    modechoicelogsums[:,dtazi] = m.logsums()
    
d.idca.MODECHOICELOGSUM[screen_idx, :] = modechoicelogsums



[10:59:29 PM]larch.Model: Model.setUp...
[10:59:29 PM]larch.Model: Model.setUp complete


## Destination Choice Data

In [16]:
d.set_alternatives( numpy.arange(1,nZones+1) )

# Due to a limitation of HDF5, we are not able to 
# give a choice_idco dictionary for models with lots of alternatives,
# as there is a hard limit on the quantity of data in the header for
# each array.  This limitation will probably be circumvented in a
# future version of Larch, but for now we must manually create and 
# store an entire idca array.
ch = numpy.zeros([d.nAllCases(), nZones ], dtype=numpy.float32)
dtaz = d.idco.DTAZ[:]
ch[range(dtaz.shape[0]), dtaz-1] = 1
d.new_idca_from_array('_choice_', ch, overwrite=True)

# We're also gonna overwrite the _avail_ array, so that
# all destinations are available for all tours.
d.new_idca_from_array('_avail_', numpy.ones([d.nAllCases(), nZones ], dtype=bool), overwrite=True)

In [17]:
# First we purge all of the OMX columns we plucked in the past
for i in omx.data:
    d.delete_data(i.name)
for i in omx.lookup:
    d.delete_data(i.name)


# Then we can attached the OMX as an external data souce, linking in
# entire rows from the skim matrixes, plus the lookups which are OTAZ-generic
d.idca.add_external_omx(omx, rowindexnode=d.idco.HOMETAZi, n_alts=nZones)

d.info(1)

<larch.DT> tmplryxra7g.h5f
┌────────────────┬───────┬─────────────────────────────────────────────────────────────────────────────────────────────┬───────┬────────────────────────────────────┐
│Variable        │dtype  │Source File                                                                                  │Shape  │Original Source                     │
├────────────────┼───────┼─────────────────────────────────────────────────────────────────────────────────────────────┼───────┼────────────────────────────────────┤
╞ idco ═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│AGE             │int64  │/var/folders/c3/sgn_gh2j3kv37txqvspbnyzw0000gn/T/tmplryxra7g.h5f                             │(6123,)│/tmp/exampville/exampville_person.h5│
│BIKETIME        │float64│/var/folders/c3/sgn_gh2j3kv37txqvspbnyzw0000gn/T/tmplryxra7g.h5f                             │(6123,)                                     │
│CARCOST         │float64│/var/folders/c3/sgn_gh2j3kv37txqvspbnyzw0000gn/T/tmplryxra7g.h5f                             │(6123,)                                     │
│DTAZ            │int64  │/var/folders/c3/sgn_gh2j3kv37txqvspbnyzw0000gn/T/tmplryxra7g.h5f                             │(6123,)│/tmp/exampville/exampville_tours.h5 │
│DTAZi           │int64  │/var/folders/c3/sgn_gh2j3kv37txqvspbnyzw0000gn/T/tmplryxra7g.h5f                             │(6123,)                                     │
│HHID            │int64  │/var/folders/c3/sgn_gh2j3kv37txqvspbnyzw0000gn/T/tmplryxra7g.h5f                             │(6123,)│/tmp/exampville/exampville_tours.h5 │
│HHSIZE          │int64  │/var/folders/c3/sgn_gh2j3kv37txqvspbnyzw0000gn/T/tmplryxra7g.h5f                             │(6123,)│/tmp/exampville/exampville_hh.h5    │
│HOMETAZ         │int64  │/var/folders/c3/sgn_gh2j3kv37txqvspbnyzw0000gn/T/tmplryxra7g.h5f                             │(6123,)│/tmp/exampville/exampville_hh.h5    │
│HOMETAZi        │int64  │/var/folders/c3/sgn_gh2j3kv37txqvspbnyzw0000gn/T/tmplryxra7g.h5f                             │(6123,)                                     │
│INCOME          │int64  │/var/folders/c3/sgn_gh2j3kv37txqvspbnyzw0000gn/T/tmplryxra7g.h5f                             │(6123,)│/tmp/exampville/exampville_hh.h5    │
│N_OTHERTOURS    │int64  │/var/folders/c3/sgn_gh2j3kv37txqvspbnyzw0000gn/T/tmplryxra7g.h5f                             │(6123,)│/tmp/exampville/exampville_person.h5│
│N_TOTALTOURS    │int64  │/var/folders/c3/sgn_gh2j3kv37txqvspbnyzw0000gn/T/tmplryxra7g.h5f                             │(6123,)│/tmp/exampville/exampville_person.h5│
│N_WORKTOURS     │int64  │/var/folders/c3/sgn_gh2j3kv37txqvspbnyzw0000gn/T/tmplryxra7g.h5f                             │(6123,)│/tmp/exampville/exampville_person.h5│
│PERSONID        │int64  │/var/folders/c3/sgn_gh2j3kv37txqvspbnyzw0000gn/T/tmplryxra7g.h5f                             │(6123,)│/tmp/exampville/exampville_tours.h5 │
│TOURMODE        │int64  │/var/folders/c3/sgn_gh2j3kv37txqvspbnyzw0000gn/T/tmplryxra7g.h5f                             │(6123,)│/tmp/exampville/exampville_tours.h5 │
│TOURPURP        │int64  │/var/folders/c3/sgn_gh2j3kv37txqvspbnyzw0000gn/T/tmplryxra7g.h5f                             │(6123,)│/tmp/exampville/exampville_tours.h5 │
│WALKTIME        │float64│/var/folders/c3/sgn_gh2j3kv37txqvspbnyzw0000gn/T/tmplryxra7g.h5f                             │(6123,)                                     │
│WORKS           │int64  │/var/folders/c3/sgn_gh2j3kv37txqvspbnyzw0000gn/T/tmplryxra7g.h5f                             │(6123,)│/tmp/exampville/exampville_person.h5│
│caseid          │int64  │/var/folders/c3/sgn_gh2j3kv37txqvspbnyzw0000gn/T/tmplryxra7g.h5f                             │(6123,)│/tmp/exampville/exampville_tours.h5 │
╞ idca ════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════

## Destination Choice Model Estimation

In [18]:
### DEST CHOICE MODEL

m = larch.Model(d)

m.title = "Exampville Work Tour Destination Choice"

from larch.util.piecewise import log_and_linear_function
dist_func = log_and_linear_function('DIST', baseparam='Distance')

m.utility.ca = (
    + P.ModeChoiceLogSum * X.MODECHOICELOGSUM
    + dist_func
#    + P.Distance * X.DIST
#    + P.Log1pDist * X("log1p(DIST)")
)

m.quantity = (
    + P("EmpRetail_HighInc") * X('EMP_RETAIL * (INCOME>50000)')
    + P("EmpNonRetail_HighInc") * X('EMP_NONRETAIL') * X("INCOME>50000")
    + P("EmpRetail_LowInc") * X('EMP_RETAIL') * X("INCOME<=50000")
    + P("EmpNonRetail_LowInc") * X('EMP_NONRETAIL') * X("INCOME<=50000")

)

m.quantity_scale = P.Theta
m.parameter.Theta(value=0.5, min=0.001, max=1.0, null_value=1.0)

m.parameter.EmpRetail_HighInc(holdfast=1, value=0)
m.parameter.EmpRetail_LowInc(holdfast=1, value=0)

m.computed_factor_figure_with_derivative(
            dist_func.evaluator1d('U'),
            max_x=20,
            header="Utility by Distance",
            xaxis_label="Freeflow Highway Distance (miles)",
            yaxis_label="Utility",
            short_header="U by Distance",
        )

m.maximize_loglike()
m.xhtml_report(filename=os.path.join(directory,'destination_choice_model.html'), cats='**')

[10:59:33 PM]larch.Model: Model.setUp...
[10:59:33 PM]larch.Model: Model.setUp complete
[10:59:33 PM]larch.Model: Using <larch.util.optimize.OptimizeTechnique SLSQP>
[10:59:34 PM]larch.Model: Optimization terminated successfully. [SLSQP]
[10:59:34 PM]larch.Model: Ending leg (success) using SLSQP at [  1.05287421e+00  -9.35968043e-04  -5.20771701e-02   8.11518365e-01   0.00000000e+00   0.00000000e+00   4.64005967e-01  -8.17847504e-01]
[10:59:34 PM]larch.Model: Preliminary Results
+--------------------+---------------+----------+------+----------+
 Parameter            Estimated Value Std Error  t-Stat Null Value 
+--------------------+---------------+----------+------+----------+
 ModeChoiceLogSum      1.053           nan        nan    0         
 Distance             -0.000936        nan        nan    0         
 logDistanceP1        -0.05208         nan        nan    0         
 Theta                 0.8115          nan        nan    1         
 EmpRetail_HighInc     0              fi

In [19]:
m.jupyter('params','ll','latest','UTILITYSPEC','PROBABILITYSPEC','DATA', 'excludedcases', 'NOTES')

Model Parameter Estimates
┌────────────────────┬───────────────┬──────────┬──────┬──────────┐
│Parameter           │Estimated Value│Std Error │t-Stat│Null Value│
├────────────────────┼───────────────┼──────────┼──────┼──────────┤
│ModeChoiceLogSum    │ 1.053         │ 0.08195  │ 12.85│ 0        │
│Distance            │-0.000936      │ 0.02497  │-0.04 │ 0        │
│logDistanceP1       │-0.05208       │ 0.1165   │-0.45 │ 0        │
│Theta               │ 0.8115        │ 0.04343  │-4.34 │ 1        │
│EmpRetail_HighInc   │ 0             │fixed value      │ 0        │
│EmpRetail_LowInc    │ 0             │fixed value      │ 0        │
│EmpNonRetail_HighInc│ 0.464         │ 0.2182   │ 2.13 │ 0        │
│EmpNonRetail_LowInc │-0.8178        │ 0.3048   │-2.68 │ 0        │
└────────────────────┴───────────────┴──────────┴──────┴──────────┘

Model Estimation Statistics
┌──────────────────────────────────┬─────────┬────────┐
│Statistic                         │Aggregate│Per Case│
├──────────────────────────────────┼─────────┼────────┤
│Number of Cases                   │       1897       │
│Log Likelihood at Convergence     │-3670.60 │-1.93   │
│Log Likelihood at Null Parameters │-4493.08 │-2.37   │
│Rho Squared w.r.t. Null Parameters│      0.183       │
└──────────────────────────────────┴─────────┴────────┘

Latest Estimation Run Statistics
┌────────────────────────────────────┬─────────────────────────────────────────────┐
│Estimation Date                     │Friday, October 28 2016, 10:59:33 PM         │
│Results                             │success                                      │
│Message                             │Optimization terminated successfully. [SLSQP]│
│Optimization Method                 │SLSQP                                        │
│Number of Iterations                │17                                           │
│Running Time│Total                  │0.362 seconds                                │
│            │setup                  │0:00.19                                      │
│            │null_likelihood        │0:00                                         │
│            │weight choice rebalance│0:00                                         │
│            │weight autorescale     │0:00                                         │
│            │optimize:SLSQP         │0:00.13                                      │
│            │parameter covariance   │0:00.04                                      │
│            │cleanup                │0:00                                         │
│Notes                               │did not automatically rescale weights        │
│Computer                            │paladin.thunderdome                          │
│Processor                           │Intel(R) Core(TM) i7-6700K CPU @ 4.00GHz     │
│Number of CPU Cores                 │8                                            │
│Number of Threads Used              │8                                            │
│Installed Memory                    │24.0 GiB                                     │
│Peak Memory Usage                   │155.36328125 MiB                             │
└────────────────────────────────────┴─────────────────────────────────────────────┘

Code,Alternative,Resolved Utility
*,all elemental alternatives,1.053*MODECHOICELOGSUM- 0.000936*DIST- 0.05208*log1p(DIST)+ 0.8115 * log( 1*EMP_RETAIL * (INCOME>50000) + 1.59*EMP_NONRETAIL*(INCOME>50000) + 1*(EMP_RETAIL*(INCOME<=50000)) + 0.4414*(EMP_NONRETAIL*(INCOME<=50000)))
Code,Alternative,Formulaic Utility
*,all elemental alternatives,ModeChoiceLogSum*MODECHOICELOGSUM+ Distance*DIST+ logDistanceP1*log1p(DIST)+ Theta * log( exp(EmpRetail_HighInc)*EMP_RETAIL * (INCOME>50000) + exp(EmpNonRetail_HighInc)*EMP_NONRETAIL*(INCOME>50000) + exp(EmpRetail_LowInc)*(EMP_RETAIL*(INCOME<=50000)) + exp(EmpNonRetail_LowInc)*(EMP_NONRETAIL*(INCOME<=50000)))


Code,Alternative,Resolved Probability
1,a1,exp(Utility[a1])/exp(Utility[ROOT])
2,a2,exp(Utility[a2])/exp(Utility[ROOT])
3,a3,exp(Utility[a3])/exp(Utility[ROOT])
4,a4,exp(Utility[a4])/exp(Utility[ROOT])
5,a5,exp(Utility[a5])/exp(Utility[ROOT])
6,a6,exp(Utility[a6])/exp(Utility[ROOT])
7,a7,exp(Utility[a7])/exp(Utility[ROOT])
8,a8,exp(Utility[a8])/exp(Utility[ROOT])
9,a9,exp(Utility[a9])/exp(Utility[ROOT])
10,a10,exp(Utility[a10])/exp(Utility[ROOT])


Code,Alternative,# Avail,# Chosen,Availability Condition
1,a1,1897,10,n/a
2,a2,1897,24,n/a
3,a3,1897,42,n/a
4,a4,1897,69,n/a
5,a5,1897,183,n/a
6,a6,1897,256,n/a
7,a7,1897,248,n/a
8,a8,1897,305,n/a
9,a9,1897,259,n/a
10,a10,1897,249,n/a


,Criteria,Data Source,# Cases Excluded,# Cases Remaining
0,TOURPURP != 1,idco,4226,1897
